# This notebook implements the BILP-Q tutorial in Cirq

In [3]:

try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")
    import cirq
import cirq_google
from Utils_CSG import *
import sympy 
import numpy as np
from typing import Sequence, Tuple
from cirq.circuits import InsertStrategy
from cirq.contrib.svg import SVGCircuit
import matplotlib.pyplot as plt
import time

AttributeError: module 'gmpy2' has no attribute 'version'

## Starting with the a given CSG
Using the same steps as in the BILP-Q Qiskit tutorial to generate the Q-matrix

In [ ]:
agents = 3
coalition_values={
    '1':30,
    '2':40,
    '3':25,
    '1,2':70,
    '1,3':60,
    '2,3':65,
    '1,2,3':90
}
coalition_values



In [ ]:
# agents = 2
# coalition_values={
#     '1':30,
#     '2':40,
#     '1,2':75

# }
# print(coalition_values)
# c,S,b = convert_to_BILP(coalition_values)       # A function in Utils_CSG.py
# print(f'c = {c}\nS = {S}\nb = {b}')

In [ ]:
c,S,b = convert_to_BILP(coalition_values)       # A function in Utils_CSG.py
print(f'c = {c}\nS = {S}\nb = {b}')

In [ ]:
qubo_penalty = 50 * -1

linear,quadratic = get_QUBO_coeffs(c,S,b,qubo_penalty)        # A function in Utils_CSG.py



In [ ]:
Q = np.zeros([len(linear),len(linear)])
Qmax = 0
#diagonal elements
for key,value in linear.items():
    if Qmax < abs(value):
        Qmax = abs(value)
        
    Q[int(key.split('_')[1]),int(key.split('_')[1])] = value

#non diagonal elements
for key,value in quadratic.items():
    if Qmax < abs(value/2):
        Qmax = abs(value/2)
    Q[int(key[0].split('_')[1]),int(key[1].split('_')[1])] = value/2
    Q[int(key[1].split('_')[1]),int(key[0].split('_')[1])] = value/2


Q = Q/Qmax

w = np.zeros(len(Q[0]))
w = -Q.sum(axis = 1)
print(w)
Q.round(1)



## Initializing the circuit

There are many other ways to create circuts. Worth mentioning is that cirq can also create grid-circuits and check, whether all two-qubits gates are only applied to adjacent qubits. This can be usefull when implementing for real quantum devices (D-Wave part).

In [ ]:
def cirq_init(n_prob):
    """Initializes a circuit of n_prob Line Qubits in equal superposition
    Return: the circuit itself and the quibits used in it"""
    circuit = cirq.Circuit() #initializing a circuit object
    qubits = cirq.LineQubit.range(n_prob) # creating n_prob qubits
    circuit.append(cirq.H(q) for q in qubits) # adding the just created qubits each with a H-Gate to the circuit
    return circuit, qubits

## Creating cost and mixer layers

In [ ]:
# the mixer layer should just be a list of X rotation gates with parameter beta on each qubit
# that will be appended to the circuit

def mixer_layer(in_circuit, beta_value: float):
    """Adds a mixer layer to circuit with parameter beta_value"""
    circuit = in_circuit.copy() 
    # without copy() circuit would still be the same instance of in_circuit
    # and mixer() would in-place change in_circuit
    
    n_qubits = len(circuit.get_independent_qubit_sets())
    circuit.append([cirq.X(q) ** (beta_value* (-2.43981781636574)) for q in qubits],  strategy=InsertStrategy.NEW_THEN_INLINE)
    # adding an X-gate to the power of beta_value to every qubit. Using an insert strategy to put them all in the same moment
    # for better readability in the diagram later
    return circuit

In [ ]:
def cost_layer(in_circuit:cirq.circuits.circuit.Circuit, gamma_value:float, Q:np.ndarray) -> cirq.circuits.circuit.Circuit:
    """Adds the cost layer defined by QUBO matrix Q to circuit with the parameter gamma_value"""
    
    
    circuit = in_circuit.copy()
    nrow = np. size(Q, 0)
    ncol = np. size(Q, 1)
    
    # in the Q matrix, diagonal entries represent the coefficients for linear terms, while the of-diagonal non-zero elements
    # are the coefficients for mixed terms. A linear term will be implemented as Z rotation gate on the resprective qubit,
    # while a mixed term is implemented as ZZ rotational gate on the 2 resprective qubits.
    # the rotations are parametrized by the gamma_value for this sprecific layer
    
    for row in range(nrow):
        # we don't need to iterrate through the whole matrix, since it's symetrical. The upper right part and diagonal is enough
        for col in range(row, ncol):
            
            if row == col:
                lin = Q[row,col]
                continue 
                # if the element is on the diagonal we will collect the coefficient to apply the single Z gate after all
                # ZZ gates are done on this qubit
            
            elif Q[row,col] != 0:
                # when we have a non-diagonal element that is not zero we append a gama_value
                # parametrized ZZ rotation gate on the resprective 2 qubits
                circuit.append(cirq.ZZ(qubits[row],qubits[col]) ** (gamma_value* 131.3328043  * Q[row,col]))
        # the single qubit z gate for linear terms at the end of all ZZ gate for this qubit 
        if lin != 0:
            circuit.append(cirq.Z(qubits[row]) ** (gamma_value  * 131.3328043 *  w[row]))
                  
    return circuit

## Creating the circuit
Note: the layer-functions still change the circuit in-place

In [ ]:
p = 1 # number of layers

# Parameters are introduced as 'symbols'. Symbols can be used like variables in a circuit but need to be 'resolved'
# (get a value assigned to) when we want to simulate or run the circuit
beta = [sympy.Symbol("β_"+str(i)) for i in range(p)]
gamma = [sympy.Symbol("γ_"+str(i)) for i in range(p)]
print(beta, gamma)

In [ ]:
problem_size = len(Q[0]) # defines how many qubits will be necessary to represent the problem 

qaoa, qubits = cirq_init(problem_size)

param_dict = {} #used to store the parameters in a dictionary. For later use in the param resolver
for i in range(p):
    qaoa = cost_layer(qaoa, gamma[i], Q)
    qaoa = mixer_layer(qaoa, beta[i])
    param_dict[gamma[i]] = 1
    param_dict[beta[i]] = 1
    
param_dict

In [ ]:
# better printing with SVG
SVGCircuit(qaoa)

In [ ]:
def state_vector_params(gamma, beta, circuit):
    simulator = cirq.Simulator() # initializing the simulator object
    params = cirq.ParamResolver({"γ_0": gamma, "β_0": beta}) 
    #the param resolver maps values from the function input to the respective variables in the circuit
    
    result = abs(simulator.simulate(circuit, param_resolver=params).final_state_vector)
    # simulates the final state vector of the circuit with the input values for the parameters gamma and beta
    # only take tha absolut values since the imaginary parts should only exist due to rounding errors
    sum = 0
    for elm in result:
        sum += elm**2
    if 1 != round(sum,3):
        print("Warning: final state vector is normalized. Difference to 1 is more than .001")
    
    return result
 


In [ ]:
def wf_energy(circuit, param_dict, M):
    simulator = cirq.Simulator() # initializing the simulator object
    params = cirq.ParamResolver(param_dict) 
    #the param resolver maps values from the function input to the respective variables in the circuit
    
    vec = abs(simulator.simulate(circuit, param_resolver=params).final_state_vector)
    return vec @ M @ vec

# Utilities

In [ ]:
def M_z(dim):
    """For measurments in the computational basis this function can create a matrix corresponding to the 
    tensor produkt of number of dim Pauli-Z matrices. 
    dim: number of qubits in the circuit"""
    mat_Z = np.eye(2)
    mat_Z[1,1] = -1
    M = [1]
    for i in range(dim):
        M = np.kron(M, mat_Z)
    return M

def ez_filter(n, dim):
    '''
    Creates matrices like Z_n (for n={1,...,#qubits}) , where we apply the Z gate to the n-th qubit
    in a circuit of dim number of qubits
    '''
    # start with Pauli-Z matrix
    ez_f = np.eye(2)
    ez_f[1,1] = -1

    if n == 1:
        #with for Z_1 we need to do the tensor product of Pauli-Z with dim-1 unit matrices
        ez_f = np.kron(ez_f, np.eye(2**(dim-1) ))
        
    else:
        #for Z_n we need a tensor product of n-1 unit matrices then Pauli-Z and again unit matrices for all the other qubits
        ez_f = np.kron(np.eye(2**(n-1)), ez_f)
        ez_f = np.kron(ez_f, np.eye(2**(dim-n)))   

    return ez_f

def m_op(Q):
    '''
    A matrix corresponding to the problem hamiltonian H_c. To compute the expectation values in terms of cost for the qubo
    '''
    dim = len(Q[1])
    
    nrow = np. size(Q, 0)
    
    ncol = np. size(Q, 1)
        
    ez = np.zeros((2**nrow, 2**ncol))

    for row in range(nrow):
        
        for col in range(row, ncol):
            
            if row == col:
                lin = Q[row,col]
               
                continue 


            elif Q[row,col] != 0:
               
                ez += Q[row,col] * ez_filter(row+1, dim) @ ez_filter(col+1, dim)


        if lin != 0:
            ez += w[row] * ez_filter(row+1, dim)

        
    return ez


def to_bin_array(n):
    ar = np.zeros(2**agents - 1)
    for i in str(bin(n))[2:]:
        ar = np.append(ar, int(i))
 

    return ar[-(2**agents-1):]




In [ ]:
# defining the measurement operator
M = m_op(Q)
#M = M_z(problem_size)
M.shape


In [ ]:
if p ==1:
    grid_size = 40
    gamma_max = 2
    beta_max = 2

    # Do the grid search.
    energies = np.zeros((grid_size, grid_size))
    for i in range(0,grid_size):
        for j in range(0,grid_size):
            vec = state_vector_params(i * gamma_max / grid_size, j * beta_max / grid_size, qaoa)
            energies[i, j] = vec @ M @ vec

    """Plot the energy as a function of the parameters 𝛄 and β found in the grid search."""
    plt.ylabel(r"$\gamma$")
    plt.xlabel(r"$\beta$")
    plt.title("Energy as a function of parameters")
    plt.imshow(energies, extent=(0, beta_max, gamma_max, 0))
    plt.colorbar()
    plt.show()
    ind = np.unravel_index(np.argmin(energies, axis=None), energies.shape) 
    ind_gam, ind_bet = ind
    eng = energies[ind]
    opt_gam = ind_gam / grid_size * gamma_max
    opt_bet = ind_bet / grid_size * beta_max
    print(f"Numerically found parameters: Gamma = {opt_gam}, Beta = {opt_bet}\nEnergy = {eng}")

In [ ]:
# Adding a measurement layer
measure_qaoa = qaoa.copy()
measure_qaoa.append(cirq.measure(qubits, key = 'result'))
simulator = cirq.Simulator()

In [ ]:
if p == 1:
    num_reps = 1000
    params = cirq.ParamResolver({'γ_0': 54.722, 'β_0': -2.439})

    measure_result = simulator.run(measure_qaoa, param_resolver=params, repetitions = num_reps)
    counts = measure_result.histogram(key='result')
    common_solution = counts.most_common(3)
    counts.most_common(10)

# Optimization with expectation

In [ ]:
# Basic gradient descent to find the best parameter for the lowest energy level
def expec_optim(param_dict, circuit, qubits, lr, epochs):
    """param_dict: dictionary with parameters {γ_0: 0, β_0: 0, .... }
       returns one step optimimized parameter dictionary
    """
    # Creating the meassurement operator
    
    
    # setting up some utilities
    opt_energy = float('inf')
    opt_param_dict = {}
    simulator = cirq.Simulator() # initializing the simulator object
    params = cirq.ParamResolver(param_dict)
    
    random_restart = 0
    vals = []
    
    
    for step in range(epochs):
        
        # The final state vector and energy of the circuit with the parameter values given by param_dict
        base_state = abs(simulator.simulate(circuit, param_resolver=params).final_state_vector)
        base_energy = base_state @ M @ base_state
        vals.append(base_energy)
        # whenever the energy is lower than previously detected, the best value and it's parameter will be stored
        if base_energy < opt_energy: 
            opt_energy = base_energy
            opt_param_dict = param_dict.copy()
        
        # Calculating the gradient numerically    
        grad = np.zeros(len(param_dict))
        j = 0
        for i in param_dict:

            param_dict[i] += 0.001
            params = cirq.ParamResolver(param_dict)
            eps_state = abs(simulator.simulate(circuit, param_resolver=params).final_state_vector)
            param_dict[i] -= 0.001
            eps = base_energy - (eps_state @ M @ eps_state)
            grad[j] = eps
            j += 1

        grad = grad * lr
        
        j = 0
        for i in param_dict:
            param_dict[i] += grad[j]
            j +=1
        
        # the energy of the learned state = base state + gradient * learning rate
        params = cirq.ParamResolver(param_dict) # new parameter after the gradient step 
        vec = abs(simulator.simulate(circuit, param_resolver=params).final_state_vector)
        energy = vec @ M @ vec #energy after the gradient step
        if energy < opt_energy:
            opt_energy = energy
            opt_param_dict = param_dict.copy()
        else: 
            random_restart +=1
        
        if random_restart == epochs//20:
            
            random_restart = 0
            param_dict[gamma[0]] += random.uniform(-2, 2)
            param_dict[beta[0]] += random.uniform(-2, 2)
            
        if not step % 25:    
            print(f"Epoch {step+1}, Energy: {opt_energy}")
    x = (range(len(vals)))
    plt.plot(x,vals)
    plt.show()
    return opt_param_dict.copy(), opt_energy


In [ ]:
# Basic gradient descent to find the best parameter for the lowest energy level
def q_cost(X):
    val = 0
    i = 0
    for prob in X:
        bits = np.array(to_bin_array(i))
        val += prob**2 * bits @ Q @ bits
        i += 1
    return val

def Q_optim(param_dict, circuit, qubits, lr, epochs):
    """param_dict: dictionary with parameters {γ_0: 0, β_0: 0, .... }
       returns one step optimimized parameter dictionary
    """
    t_start = time.time()
    
    
    # setting up some utilities
    opt_cost = float('inf')
    opt_param_dict = {}
    simulator = cirq.Simulator() # initializing the simulator object
    params = cirq.ParamResolver(param_dict)
    
    random_restart = 0
    vals = []
    
    
    for step in range(epochs):
        
        # The final state vector and energy of the circuit with the parameter values given by param_dict
        base_state = abs(simulator.simulate(circuit, param_resolver=params).final_state_vector)
        base_cost = q_cost(base_state)
        vals.append(base_cost)
        # whenever the energy is lower than previously detected, the best value and it's parameter will be stored
        if base_cost < opt_cost: 
            opt_cost = base_cost
            opt_param_dict = param_dict
        
        # Calculating the gradient numerically    
        grad = np.zeros(len(param_dict))
        j = 0
        for i in param_dict:

            param_dict[i] += 0.0001
            params = cirq.ParamResolver(param_dict)
            eps_cost = q_cost(abs(simulator.simulate(circuit, param_resolver=params).final_state_vector))
            param_dict[i] -= 0.0001
            eps = base_cost - eps_cost
            grad[j] = eps
            j += 1

        grad = grad * lr
        
        j = 0
        for i in param_dict:
            param_dict[i] += grad[j]
            j +=1
        
        # the energy of the learned state = base state + gradient * learning rate
        params = cirq.ParamResolver(param_dict) # new parameter after the gradient step 
        vec = abs(simulator.simulate(circuit, param_resolver=params).final_state_vector)
        grad_cost = q_cost(vec) #energy after the gradient step
        if grad_cost < opt_cost:
            opt_cost = grad_cost
            opt_param_dict = param_dict
        
        if grad_cost >= base_cost: 
            random_restart +=1
        elif abs(base_cost - grad_cost) < 0.00001:
            print("Grad vanished")
            random_restart = epochs//20
        
        
        if random_restart >= epochs//20:
            print("Restart")
            random_restart = 0
            for par in param_dict:
                param_dict[par] += random.uniform(-5, 5)
            
        if not step % 25:    
            print(f"Epoch {step+1}, Energy: {opt_cost}")
        t_end = time.time()
        last_epoch = step
        
        if t_end - t_start > 600:
            print("More than 600s runtime")
            break
        
    x = (range(len(vals)))
    plt.plot(x,vals)
    plt.show()
    
    print(f"Time elapsed: {round(t_end - t_start,2)}s")
    print(f"Time per epoch: {round((t_end - t_start)/last_epoch,4)}s")
    return opt_param_dict, opt_cost

In [ ]:
solutions = []


for i in range(p):
    param_dict[gamma[i]] = 1
    param_dict[beta[i]] = 1
print(param_dict)
solutions.append(param_optim(param_dict, qaoa, qubits, 0.2, 600))
solutions

In [ ]:
opt_params = dict()
opt_params = solutions[0][0].copy()

opt_params

In [ ]:
num_reps = 1000
params = cirq.ParamResolver(opt_params)
print(params)
measure_result = simulator.run(measure_qaoa, param_resolver=params, repetitions = num_reps)
cirq.plot_state_histogram(measure_result, plt.subplot())
plt.show()
counts = measure_result.histogram(key='result')
solution = counts.most_common(10)
solution

In [ ]:
default_params = dict()
for i in range(p):
    default_params[gamma[i]] = 1
    default_params[beta[i]] = 1
print(default_params)
print(opt_params)

default_en = wf_energy(qaoa, default_params, M)
opt_en = wf_energy(qaoa, opt_params, M)
print(default_en)
print(opt_en)
opt_en < default_en

In [ ]:
def find_sol(sol_counter):
    min_cost = float("inf")
    tot = 0
    for k in sol_counter:
        v = sol_counter[k]
        tot += v
        bits = np.array(to_bin_array(k))
        cost_sol =  bits @ Q @ bits
        if cost_sol < min_cost:
            min_cost = cost_sol
            sol = k
        
  
    prob = sol_counter[sol]/tot
    sol_coalition = decode(np.array(to_bin_array(sol)), coalition_values)
    return sol_coalition, sol, prob
        

In [ ]:
most_common = solution[0][0]
print("Most frequent measurment: ", most_common)
decode(to_bin_array(most_common), coalition_values)

## Solution

In [ ]:
sol, num, prob = find_sol(counts)
print(f"Best coalition: {sol}\nProbabilty: {prob}")
num

# Sampling approach

In [ ]:
simulator = cirq.Simulator()
batch_size = 100
params = cirq.ParamResolver(opt_params)

sample_results = simulator.run(measure_qaoa, param_resolver=params, repetitions = num_reps)
sample_results = sample_results.histogram(key='result')
#sample_results

In [ ]:
def cost(sample_results):
    '''calculates the accumulated cost xQx for all x in samples_results'''
    cost = 0
    num = 0
    for k in sample_results:
        bits = np.array(to_bin_array(k))
        num += sample_results[k]
        cost += sample_results[k] * bits @ Q @ bits
        
    return cost/num
    
cost(sample_results)

bit = np.array(to_bin_array(36))
bit @ Q @ bit



In [ ]:
np.array(to_bin_array(28))


In [ ]:
def sample_optimizer(param_dict, circuit, qubits, lr, batch_size = 1000, epochs = 300):
    """param_dict: dictionary with parameters {γ_0: 0, β_0: 0, .... }
       returns one step optimimized parameter dictionary
    """
    # Creating the meassurement operator
    
    vals = []
    # setting up some utilities
    opt_cost = float('inf')
    opt_param_dict = {}
    simulator = cirq.Simulator() # initializing the simulator object
    params = cirq.ParamResolver(param_dict)
    
    random_restart = 0
    
    
    
    for step in range(epochs):
        
        # get cost from samples
        sample_results = simulator.run(circuit, param_resolver=param_dict, repetitions = batch_size)
        sample_results = sample_results.histogram(key='result')
        
        base_cost = cost(sample_results)
        vals.append(base_cost)
        # whenever the energy is lower than previously detected, the best value and it's parameter will be stored
        if base_cost < opt_cost: 
            opt_cost = base_cost
            opt_param_dict = param_dict
        
        # Calculating the gradient numerically    
        grad = np.zeros(len(param_dict))
        j = 0
        for i in param_dict:

            param_dict[i] += 0.1
            params = cirq.ParamResolver(param_dict)
            eps_results = simulator.run(circuit, param_resolver=param_dict, repetitions = batch_size)
            eps_results = eps_results.histogram(key='result')
            param_dict[i] -= 0.1
            eps = base_cost - cost(eps_results)
            grad[j] = eps
            j += 1

        grad = grad * lr
        
        j = 0
        for i in param_dict:
            param_dict[i] += grad[j]
            j +=1
        
        # the energy of the learned state = base state + gradient * learning rate
        params = cirq.ParamResolver(param_dict) # new parameter after the gradient step 
        grad_results = simulator.run(circuit, param_resolver=param_dict, repetitions = batch_size)
        grad_results = grad_results.histogram(key='result')
        
        # cost after grad step
        grad_cost = cost(grad_results)
        if grad_cost > base_cost:
            random_restart += 1
               
        if grad_cost < opt_cost:
            opt_cost = grad_cost
            opt_param_dict = param_dict
        
            
        
        if random_restart > 10:
            print(f"restart at {step}")
            random_restart = 0
            param_dict[gamma[0]] += random.uniform(-2, 2)
            param_dict[beta[0]] += random.uniform(-2, 2)
            
        if not step % 50:    
            print(f"Epoch {step+1}, Cost: {grad_cost}, Grad: {grad}")
        
    x = (range(len(vals)))
    plt.plot(x,vals)
    plt.show()    
    return opt_param_dict.copy(), opt_cost


In [ ]:
samples_sol = []
samples_sol.append(sample_optimizer(param_dict, measure_qaoa, qubits, lr = 1* 10**(-3),  batch_size =1000, epochs = 500))
samples_sol

In [ ]:
opt_params = dict()
opt_params = samples_sol[0][0]

for i in range(p):
    param_dict[gamma[i]] = 1
    param_dict[beta[i]] = 1

num_reps = 1000
params = cirq.ParamResolver(opt_params)
print(params)
measure_result = simulator.run(measure_qaoa, param_resolver=params, repetitions = num_reps)
cirq.plot_state_histogram(measure_result, plt.subplot())
plt.show()
counts = measure_result.histogram(key='result')
solution = counts.most_common(10)
solution


In [ ]:
def expEnergy(circuit, M, param_dict = None):
    if param_dict != None:
        params = cirq.ParamResolver(param_dict)
    vec = abs(simulator.simulate(circuit, param_resolver=params).final_state_vector)
    return vec @ M @ vec

In [ ]:
en_params = opt_params.copy()
print(en_params)
expEnergy(qaoa, M, en_params)

In [ ]:
opt_params

# Solution

In [ ]:
sol, num, prob = find_sol(counts)
print(f"Best coalition: {sol}\nProbabilty: {prob}")
num

In [ ]:
def show_cost():
    op = float('inf')
    vals = []
    for i in range(128):
        bits = np.array(to_bin_array(i))
        co = bits @ Q @ bits
        vals.append(co)
        if co < op:
            op = co
            index = i

    print(index, op)

    x = range(len(vals))
    plt.plot(x,vals)
    return vals

v = show_cost()

revise optimization
simulator to non noise
2 agents